
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Modele generatywne - VAE</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Modele generatywne
* autoenkodery mają za zadanie __wyłącznie__ dobrze odtwarzać
  * dane w latent __nie mają__ żadnego rozkładu i nie da się generować
* co to znaczy generować?
  * model ma się nauczyć rozkładu danych (prawdziwy, ale nieznany) $P_X$
  * musi być możliwość generowania z zadanego rozkładu $P_Z$ i dekodowania do $P_X$

## Kullback-Leibler
$KL(p\mid\mid q)\; \text{to prawie} -\sum_x q(x)\log q(x)- (-\sum_x p(x)\log p(x))=H(p)-H(q)$

Co jest __prawie__ dokładnie mówiące, że $KL(p\mid\mid q)=0$ jest równe $0$ jeśli $p$ niesie tyle samo informacji ile niesie $q$

Prawie, bo KL jest niesymetryczne i odnosi się do $p$, więc powinno być __pod warunkiem $p$__

Stąd 
$$\begin{align*}
KL(p\mid\mid q)&=-\sum_x p(x)\log q(x)+\sum_x p(x)\log p(x))\\
&=\sum_xp(x)\log\frac{p(x)}{q(x)}=-\sum_xp(x)\log\frac{q(x)}{p(x)}
\end{align*}$$

1. $KL(p\mid\mid q)\geq0$
2. $KL(p\mid\mid q)\neq KL(q\mid\mid p)$, więc __nie jest__ metryką

$KL$ jest miarą podobieństwa rozkładów

## Variational Auto-Encoder VAE
* model składa się z 
  * enkodera $E:X\longrightarrow Z$
  * dekodera (generatora) $D:Z\longrightarrow X$
  
1. potrzebujemy wyliczyć $$p(z\mid x)=\frac{p(x\mid z)p(z)}{p(x)}$$
 jednak wyliczenie $p(x)$ jest trudne (ang. intractable)
  * metodami Monte-Carlo
    * pracochłonne
    * wysoka wariancja, zerowy bias
  * wariacyjnymi
    * zerowa wariancja, wysoki bias
2. __pomysł__: weźmy jakieś $q(z)$, które
  * jest możliwe do policzenia
  * niech $q(z)$ przypomina $q(z\mid x)$
  * niech miarą podobieństwa będzie $KL()$
3. obliczamy odległość (prawie)
$$\begin{align}
KL(q(z)\mid\mid p(z\mid x))&=-\sum q(z)\log\frac{p(z\mid x)}{q(z)}\\
&\hskip5em\text{ale}\; p(z\mid x)=\frac{p(x,z)}{p(x)}\\
&=-\sum q(z)\log\frac{p(x,z)/p(x)}{q(z)}\\
&\hskip5em\text{rozpisując logarytm ilorazu}\\
&=-\sum q(z)\left[\log\frac{p(x,z)}{q(z)}-\log p(x)\right]\\
&=-\sum q(z)\log\frac{p(x,z)}{q(z)}+\sum q(z)\log p(x)\\
&\hskip5em\text{sumowanie jest po $z$, więc $p(x)$ można wyciągnąć}\\
&=-\sum q(z)\log\frac{p(x,z)}{q(z)}+p(x)\sum q(z)\\
&\hskip5em\text{ale}\;\sum q(z)=1\\
&=-\sum q(z)\log\frac{p(x,z)}{q(z)}+p(x)
\end{align}$$


3. Teraz zamieńmy kolejność w równaniu
$$
p(x)=KL(q(z)\mid\mid p(z\mid x))+\underbrace{\sum q(z)\log\frac{p(x,z)}{q(z)}}_{L}
$$
4. $x$ jest punktem, więc $p(x)=const$
  * chcemy __minimalizować__ $KL(q(z)\mid\mid p(z\mid x))$
  * co jest wobec tego osiągalne przez __maksymalizację__ $L$
  * $L$ to variational lower bound
5. jak jednak maksymalizować $L$???

$$\begin{align}
L&=\sum q(z)\log\frac{p(x,z)}{q(z)}\\
&\hskip5em\text{zastępujemy $p(x,z)$ definicją $p(x\mid z)p(z)$}\\
&\hskip5em\text{ponieważ potrzebna jest definicja $px\mid z)$ jako dekodera}\\
&=\sum q(z)\log\frac{p(x\mid z)p(z)}{q(z)}\\
&\hskip5em\text{znowu rozpisując logarytm iloczynu}\\
&=\sum q(z)\left[\log p(x\mid z) +\log\frac{p(z)}{q(z)}\right]\\
&=\sum q(z)\log p(x\mid z) +\sum q(z)\log\frac{p(z)}{q(z)}\\
&\hskip5em\text{pierwszy składnik to po prostu wartość oczekiwana po wartościach $q(z)$}\\
&=\mathbb{E}_{q(z)}\log p(x\mid z) +\sum q(z)\log\frac{p(z)}{q(z)}\\
&\hskip5em\text{a drugi to wartość miary $-KL(q(z)\mid\mid \log p(z))$}\\
&=\mathbb{E}_{q(z)}\log p(x\mid z) -KL(q(z)\mid\mid \log p(z))\\
&\hskip5em\text{$q(z)$ jest funkcją realizowaną przez enkoder na podstawie danych $x\in ~X$}\\
&=\mathbb{E}_{q(z)}\log p(x\mid z) -KL(q(z\mid x)\mid\mid \log p(z))
\end{align}$$

## Variational Auto-Encoder VAE 
W modelu VAE uczenie będzie polegało na __maksymalizowaniu__
$$L=\mathbb{E}_{q(z\mid x)}\log p(x\mid z)-KL(q(z\mid x)\mid\mid p(z))$$
1. pierwszy składnik odpowiada w rzeczywistości błędowi rekonstrukcji
  * niech założony prior $p(z)$ będzie rozkładem normalnym $N(0,1)$
  * w rozkładzie normalnym mamy czynnik $\exp(-(x-\hat{x})^2)$
  * biorąc logarytm zostaje nam dokładnie $(x-\hat{x})^2$
2. drugi składnik jest odpowiedzialny za to, by rozkład warunkowy $q(z\mid x)$ był jak najbardziej podobny do założonego rozkładu prior $p(z)$
3. model składa się z 
  * __enkodera__ realizujacego $q(z\mid x)$
  * dekodera realizującego $p(x\mid z)$
    * $z$ nie jest dowolne, lez zależy od $x$
4. dekoder jest deterministyczny
5. enkoder jest stochastyczny i zwraca wszystkie parametry założonego rozkładu prior
  * dla $\mathcal{N}(0,\mathbb{I})$ są to dwa parametry $\mu, \Sigma$
    * $\mu$ jest $D$ wymiarowy
    * $\Sigma$ jest $D\times D$
    * $\mathbb{I}$ zakładamy jako macierz przekątniową dla prostoty i symetrii (i obliczalności), enkoder zwraca drugi $D$ wymiarowy partametr
  * po wyliczeniu $(\mu, \sigma)$ wartość $z$ jest __samplowana__ z wyliczonego rozkładu

## Operacje w przestrzeni latent

### kodowanie i dekodowanie danych wejściowych
* ze zbioru uczącego, ze zbioru testującego
* miarą jest wartość błędu odtwarzania

### samplowanie z okolic
* kodujemy przykład $x$
* samplujemy z okolic
  * enkoder zwraca __rozkład__ podany przez parametry
  * dla rozkładu normalnego samplujemy z tego rozkładu o wyliczonych parametrach $z=(\mu,\sigma^2)$ i dekodujemy (dekoder jest deterministyczny!)
* wygenerowane przykłady powinny należeć do przykładów z rozkładu $X$
* wygenerowane przykłady powinny być __podobne__ do oryginalnego
  * co to znaczy _podobne_?
  * podobne zdjęcia, częsty problem z dobrą oceną
  * przykłady mogą pochodzić ze zbiorów o małej wariancji danych, np. twarze (CelebA)
  * niezależne miary, np. biblioteki dla oceny grafów z danej klasy

### interpolacja
* interesuje nas przejście między dwoma przykładami
* kodujemy $x_1$ i $x_2$ dostając $z_1, z_2$
* interpolacja między nimi
* liniowa?
* paradoks "mydlanej bańki"
  * niech $P_Z$ $D$-wymiarowym rozkładem normalnym $\mathcal{N}(0,\mathbb{I})$
  * losując $z\sim P(Z)$ losujemy $D$ zmiennych $z_i\sim\mathcal{N}(0,1)$ _niezależnie_
  * kwadrat normy $Y=z_1^2+z_2^2+\ldots+z_D^2$
  * zmienna losowa $Y$ ma rozkład $\chi^2$
  * dla $D>30$ korzystamy w przybliżeniu z rozkładu normalnego $\mathcal{N}(\sqrt{2D-1},1)$
  * a więc normy wszystkich przykładów dla np. $D=100$ mają wartość oczekiwaną rzędu $14$!
  * oznacza to, że wszystkie $z\sim p(z)$ leżą na bańce o niewielkiej grubości
    * grubość jest stała, więc stosunkowo coraz węższa w stosunku do średnicy
  * punkty interpolacji liniowej przechodzą przez obszar o małej masie prawdopodobieństwa
* jak sobie z tym poradzić w interpolacjach?
  * interpolacje sferyczne
    * heureza oparta na paradoksie bańki
  * inny rozkład
    * weźmy model o ustalonym prior $p(z)$
    * losujemy dwa $z$, obliczamy punkt w środku $z=0.5z_1+0.5z_2$
    * jaki będzie rozkład punktów środkowych?
    * dla rozkładów prior o znanej średniej, rozkład środkowych będzie __inny__
    * uczenie i interpolacje będą na innych rozkładach, stąd interpolacje nie mogą być poprawne
    * potrzebny rozkład prior, dla którego średnia nie będzie się różnić
    * takim rozkładem jest np. rozkład Cauchy-ego
    $$C(x)=\frac{1}{\pi}\frac{\lambda}{\lambda^2+(x-\alpha)^2}$$
      * $\alpha$ to parametr położenia, $\lambda>0$ skali
    * to tzw. rozkład niewidomego łucznika
    * rozkład Cauchy-ego __nie ma__ średniej, wariancji, ani wyższych momentów
    * rozkład średnich $z\sim C$ jest identyczny z $C$
    * można nauczyć model biorąc $p(z)\sim C$
    * jednak $C$ ma "grube" ogony i samplowanie stamtąd daje nienajlepsze wyniki
    * inne rozwiązanie 
      * użyć normalny prior do uczenia
      * przy interpolacji
        * wylosować dwa punkty z normalnego
        * przez funkcje $CDF$ rozkładu normalnego i rozkład Cauchy przeprowadzić je do przestrzeni rozkładu Cauchy
        * przeprowadzić interpolację liniową
        * wykonać odwzorowanie odwrotne
      * możliwe, dające ten sam rezultat, rozwiązanie przenoszące rozkład punktów średnich do założonego rozkładu wykorzystując tzw. earth-moving (OT)
      
      
<img src="../nn_figures/automata-manifold.pdf" width="100%"> 